# One-Dimension Line Search 一维线搜索

## 问题描述

Consider a rectangular wing of span $b$ and chord $c$. Its planform area is thus $ S=b c $ and its aspect ratio is $ A = b^{2} / S $. The drag of this wing can be approximated as,
$$
C_{D}=k C_{f} \frac{S_{w e t}}{S}+\frac{C_{L}^{2}}{\pi A e}
$$
The first term corresponds to the parasite drag. $ C_{f} $ is the skin friction coefficient, which for a fully turbulent boundary layer can be approximated as,
$$
C_{f}=\frac{0.074}{R e^{0.2}}
$$
Here, the Reynolds number $ (R e=\rho V c / \mu) $ is based on the wing chord. $ k $ is the form factor, which accounts for the effects of pressure drag.

The second term in Equation (1) is the induced drag, where $ e $ is the Oswald efficiency factor. The lift coefficient $ C_{L} $ and the wing planform area $ S $ are to be kept constant. The values for all the constants are listed in Table 1. 

1. Write the total drag coefficient as a function of $ A $.
2. Minimize $ C_{D} $ with respect to $ A $ using:   
    (a) The golden section method  
    (b) A line search method that satisfies sufficient decrease. (Bonus: A line search that satisfies the strong Wolfe conditions.)  
    Converge the solutions to 6 significant digits.  
3. Discuss the relative performance of these two methods. Try different starting points/intervals and compare convergence rates, number of iterations and any other metrics you find suitable.

\begin{array}{lrll}
\text { Quantity } & \text { Value } & \text { Units } & \text { Description } \\
\hline \rho & 1.23 & \mathrm{~kg} / \mathrm{m}^{3} & \text { density of air } \\
\mu & 17.8 \times 10^{-6} & \mathrm{~kg} /(\mathrm{m} \mathrm{sec}) & \text { viscosity of air } \\
V & 35 & \mathrm{~m} / \mathrm{s} & \text { airspeed } \\
S & 11.8 & \mathrm{~m}^{2} & \text { planform area } \\
S_{\text {wet }} & 2.05 S & \mathrm{~m}^{2} & \text { wing wetted area } \\
k & 1.2 & & \text { form factor } \\
C_{L} & 0.3 & & \text { lift coefficient } \\
e & 0.96 & & \text { Oswald efficiency factor } \\
\hline
\end{array}


In [2]:
import numpy as np

def Cal_Re(c, rho=1.23, V=35.0, mu=17.8e-6):
    """Calculate Reynolds number."""
    return rho*V*c/mu

def Cal_Cf(Re):
    """Calculate the skin friction coefficient"""
    return 0.074/(Re**0.2)

def Cal_Cd(A, k=1.2, S=11.8, Sratio=2.05, Cl=0.3, e=0.96):
    """Calculate the total drag coefficient as the function of A"""
    b = np.sqrt(A*S)
    c = S/b
    Re = Cal_Re(c)
    Cf = Cal_Cf(Re)
    return k*Cf*Sratio+Cl**2/(np.pi*A*e)    

## The Golden Section Method 黄金切割法

In [19]:
class GoldenSectionMethod():
    """Golden Section Method"""
    def __init__(self, function, x_min=0.0, x_max=100.0, tau=0.618, error = 1e-6):
        self.x_max = x_max
        self.x_min = x_min
        self.function = function
        self.tau = tau
        self.error = error

    def Search(self, iter_max=2000):
        b = self.x_max
        a = self.x_min
        n = a + (b-a)*self.tau
        m = b - (b-a)*self.tau
        iteration = 1
        min_index_old = 100000

        while(iteration<=iter_max):
            if( self.function(m) <= self.function(n)):
                min_index = m
                b = n
            else:
                min_index = n 
                a = m
            n = a + (b-a)*self.tau
            m = b - (b-a)*self.tau 

            if(abs(min_index - min_index_old)<=self.error):
                print("The accutry has reached "+str(self.error))
                print('Iteration = '+str(iteration))
                break

            min_index_old = min_index
            iteration += 1

        return min_index, self.function(min_index)

Golden = GoldenSectionMethod(function=Cal_Cd)
A_min, Cd_min = Golden.Search()
print(A_min,Cd_min)

The accutry has reached 1e-06
Iteration = 21
28.394872656148546 0.011560688987446468


## A line search that satisfies the strong Wolfe conditions.